In [2]:
from creds import HOST_NAME, DB_NAME, USERNAME , PASSWORD 
import pandas as pd
import duckdb as duck
import psycopg
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def exec_query_os(query):
    conn = psycopg.connect(
        dbname=DB_NAME, user=USERNAME, password=PASSWORD, host=HOST_NAME, port="5432"
    )
    df = pd.read_sql(query, conn)

    conn.close()
    return df

def df_to_parquet(df, table_name):
    df.to_parquet(f"./data/{table_name}.parquet", compression='gzip')


## Reading NBA Data from db

In [29]:
game_box_df = exec_query_os("""
                SELECT *
                FROM public."NBA_GAME_BOX_HIST"
                where season = 2024;
            """)

game_pbp_df = exec_query_os("""
                SELECT *
                FROM public."NBA_PBP_HIST"
                where season = 2024;
            """)

player_box_df = exec_query_os("""
                SELECT *
                FROM public."NBA_PLAYER_BOX_HIST"
                where season = 2024;
            """)


In [ ]:
print(game_box_df.shape)
print(game_pbp_df.shape)
print(player_box_df.shape)


## Cleaning and Normalizing Data

### Team Game Stats

In [ ]:
team_schedules = game_box_df.copy()

# Creating a home away 'indicator' - mainly for tooltip
team_schedules['game_title'] = np.where(team_schedules['team_home_away'] == 'home',
                                      team_schedules['opponent_team_abbreviation'] + ' @ ' + team_schedules['team_abbreviation'],
                                      team_schedules['team_abbreviation'] + ' @ ' + team_schedules['opponent_team_abbreviation']
                                    )

# Creating win loss 'indicator' - mainly for tooltip
team_schedules['outcome_title'] = np.where(team_schedules['team_winner'] == True,
                                      'W ' + team_schedules['team_score'].astype(str) + '-' + team_schedules['opponent_team_score'].astype(str),
                                      'L ' + team_schedules['opponent_team_score'].astype(str) + '-' + team_schedules['team_score'].astype(str)
                                    )

print(team_schedules.shape)
team_schedules.head()

In [ ]:
# normalizing team game dataframe
# 4 final dataframes

# splitting team info into separate table
team_lkp = team_schedules.copy()
team_lkp = team_lkp[['team_id', 'team_slug', 'team_location', 'team_name',
       'team_abbreviation', 'team_display_name', 'team_short_display_name',
       'team_color', 'team_alternate_color', 'team_logo']].drop_duplicates().reset_index(drop=True)

# splitting game outcome into separate table
game_overview = team_schedules.copy()
game_overview = game_overview[['game_id', 'season', 'season_type', 'game_date', 'game_date_time', 'game_title']].drop_duplicates().reset_index(drop=True)

# splitting home and away into separate table
home_stats = team_schedules.copy()
home_stats = home_stats[home_stats['team_home_away'] == 'home'][['game_id', 'team_id',
                                                               'team_score', 'team_winner', 'assists', 'blocks', 
                                                               'defensive_rebounds', 'fast_break_points', 'field_goal_pct', 
                                                               'field_goals_made', 'field_goals_attempted', 'flagrant_fouls', 
                                                               'fouls', 'free_throw_pct','free_throws_made', 'free_throws_attempted', 
                                                               'offensive_rebounds','points_in_paint', 'steals', 'team_turnovers', 
                                                               'technical_fouls','three_point_field_goal_pct', 'three_point_field_goals_made',
                                                               'three_point_field_goals_attempted', 'total_rebounds','total_technical_fouls', 
                                                               'total_turnovers', 'turnover_points', 'turnovers']].reset_index(drop=True)

away_stats = team_schedules.copy()
away_stats = away_stats[away_stats['team_home_away'] == 'away'][['game_id', 'team_id',
                                                               'team_score', 'team_winner', 'assists', 'blocks', 
                                                               'defensive_rebounds', 'fast_break_points', 'field_goal_pct', 
                                                               'field_goals_made', 'field_goals_attempted', 'flagrant_fouls', 
                                                               'fouls', 'free_throw_pct','free_throws_made', 'free_throws_attempted', 
                                                               'offensive_rebounds','points_in_paint', 'steals', 'team_turnovers', 
                                                               'technical_fouls','three_point_field_goal_pct', 'three_point_field_goals_made',
                                                               'three_point_field_goals_attempted', 'total_rebounds','total_technical_fouls', 
                                                               'total_turnovers', 'turnover_points', 'turnovers']].reset_index(drop=True)
# adding team id's to game overview table
final_game_overview = pd.merge(
       pd.merge(
              game_overview, home_stats[['game_id', 'team_id']], on='game_id', how='left'
       ).rename(columns={'team_id':'home_team_id'}),
       away_stats[['game_id', 'team_id']], on='game_id', how='left'
       ).rename(columns={'team_id':'away_team_id'}).reset_index(drop=True)

print(f"Team Lookup Table Shape: {team_lkp.shape}")
print(f"Game Overview Table Shape: {final_game_overview.shape}")
print(f"Home Team Stats Table Shape: {home_stats.shape}")
print(f"Away Team Stats Table Shape: {away_stats.shape}")

### Player Game Stats

In [ ]:
# Creating a player info table
player_lkp = player_box_df.copy()
player_lkp = player_lkp[['athlete_id', 'athlete_display_name', 'athlete_headshot_href',
                               'athlete_position_name', 'athlete_position_abbreviation']].drop_duplicates().reset_index(drop=True)

# Creating a link to team
player_team_lkp = player_box_df.copy()
player_team_lkp = player_team_lkp[['athlete_id','team_id', 'game_date']].drop_duplicates()
player_team_lkp = player_team_lkp.groupby(['athlete_id','team_id'], as_index=False).agg({
                'game_date': ['min', 'max']
})
player_team_lkp.columns = ['athlete_id', 'team_id', 'first_game_date', 'last_game_date']
player_team_lkp = player_team_lkp.reset_index(drop=True)

# game player stat
player_game_stat= player_box_df.copy()
player_game_stat = player_game_stat[['game_id','athlete_id', 'minutes', 'field_goals_made', 'field_goals_attempted', 
                                     'three_point_field_goals_made', 'three_point_field_goals_attempted', 'free_throws_made',
                                     'free_throws_attempted', 'offensive_rebounds', 'defensive_rebounds', 'rebounds', 'assists',
                                     'steals', 'blocks', 'turnovers', 'fouls', 'plus_minus', 'points', 'starter', 'ejected',
                                     'did_not_play', 'active', 'reason']].drop_duplicates().reset_index(drop=True)

print(f"Player Lookup Table Shape: {player_lkp.shape}")
print(f"player Team Info Table Shape: {player_team_lkp.shape}")
print(f"Game Player Stats Table Shape: {player_game_stat.shape}")

### Play by Play Player Stats

In [ ]:
# Normalizing player pbp data

# grouping by event types
type_1 = [7, 8, 9, 10, 11, 12, 13, 22, 24, 25, 28, 29, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 44, 45, 46, 47, 48, 257]
type_2 = [16, 213, 214, 215, 216, 277, 278, 279, 280, 402, 412, 517, 584, 615]
type_3 = [61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 84, 86, 87, 88, 90, 206]
type_4 = [91, 92, 93, 94, 95, 96, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
          126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
          148, 149, 150, 151, 152, 153]
type_5 = [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 157, 165, 166]
type_6 = [155, 156]

game_pbp_frmt = game_pbp_df.copy()
game_pbp_frmt['period_id'] = game_pbp_frmt['game_id'].astype(str) + game_pbp_frmt['period_number'].astype(str)

type_group_conditions = [
            game_pbp_frmt['type_id'] == 0,
            game_pbp_frmt['type_id'].isin(type_1),
            game_pbp_frmt['type_id'].isin(type_2),
            game_pbp_frmt['type_id'].isin(type_3),
            game_pbp_frmt['type_id'].isin(type_4),
            game_pbp_frmt['type_id'].isin(type_5),
            game_pbp_frmt['type_id'].isin(type_6)
]
game_pbp_frmt['type_group_id'] = np.select(type_group_conditions, [0, 1, 2, 3, 4, 5, 6], default=0)

type_group_desc_conditions = [
            game_pbp_frmt['type_group_id'] == 0,
            game_pbp_frmt['type_group_id'] == 1,
            game_pbp_frmt['type_group_id'] == 2,
            game_pbp_frmt['type_group_id'] == 3,
            game_pbp_frmt['type_group_id'] == 4,
            game_pbp_frmt['type_group_id'] == 5,
            game_pbp_frmt['type_group_id'] == 6
]
type_group_descs = ['None', 'foul', 'pause', 'turnover', 'shot', 'free-throw', 'rebound']
game_pbp_frmt['type_group_text'] = np.select(type_group_desc_conditions, type_group_descs, default='None')

# game period lookup
game_period_lkp = game_pbp_frmt.copy()
game_period_lkp = game_period_lkp[['game_id', 'period_id', 'period_number']].drop_duplicates().sort_values(
                    by=['game_id', 'period_id'], ascending=True
                ).reset_index(drop=True)

# period time info lookup
period_time = game_pbp_frmt.copy()
period_time = period_time[['period_id', 'id', 'sequence_number', 'wallclock',
                           'clock_display_value', 'start_quarter_seconds_remaining', 'start_half_seconds_remaining',
                           'start_game_seconds_remaining', 'game_play_number', 'end_quarter_seconds_remaining', 'end_half_seconds_remaining',
                           'end_game_seconds_remaining']].drop_duplicates().reset_index(drop=True)

# game score lookup
game_score_lkp = game_pbp_frmt.copy()
game_score_lkp = game_score_lkp[['period_id', 'id', 'sequence_number',
                                 'home_score', 'away_score']].drop_duplicates().reset_index(drop=True)

# event type lookup
event_type_lkp = game_pbp_frmt.copy()
event_type_lkp = event_type_lkp[['type_group_id', 'type_group_text',
                                 'type_id', 'type_text']].drop_duplicates().reset_index(drop=True)

# play overview 
play_overview = game_pbp_frmt.copy()
play_overview = play_overview[['id', 'type_id', 'team_id', 'scoring_play', 'score_value', 'shooting_play',
                               'athlete_id_1', 'athlete_id_2', 'athlete_id_3', 'coordinate_x',
                               'coordinate_y']].drop_duplicates().reset_index(drop=True)

print(f"Game Period Lookup Table Shape: {game_period_lkp.shape}")
print(f"Period Time Info Table Shape: {period_time.shape}")
print(f"Game Score Lookup Table Shape: {game_score_lkp.shape}")
print(f"Event Type Lookup Table Shape: {event_type_lkp.shape}")
print(f"Player Play Stat Table Shape: {play_overview.shape}")

## Change Data Types and Export to parquet

In [60]:
# Adjusting data types according to ERD

GAME_PERIOD_LKP = game_period_lkp.copy()
GAME_PERIOD_LKP['period_id'] = GAME_PERIOD_LKP['period_id'].astype(int)

PERIOD_INFO_LKP = period_time.copy()
PERIOD_INFO_LKP['period_id'] = PERIOD_INFO_LKP['period_id'].astype(int)
PERIOD_INFO_LKP['id'] = PERIOD_INFO_LKP['id'].astype(int)
PERIOD_INFO_LKP['sequence_number'] = PERIOD_INFO_LKP['sequence_number'].astype(int)
PERIOD_INFO_LKP['wallclock'] = pd.to_datetime(PERIOD_INFO_LKP['wallclock'])
PERIOD_INFO_LKP['clock_display_value'] = PERIOD_INFO_LKP['clock_display_value'].astype(str)

GAME_PLAY_SCORE_LKP = game_score_lkp.copy()
GAME_PLAY_SCORE_LKP['period_id'] = GAME_PLAY_SCORE_LKP['period_id'].astype(int)
GAME_PLAY_SCORE_LKP['id'] = GAME_PLAY_SCORE_LKP['id'].astype(int)
GAME_PLAY_SCORE_LKP['sequence_number'] = GAME_PLAY_SCORE_LKP['sequence_number'].astype(int)

PLAY_TYPE_LKP = event_type_lkp.copy()
PLAY_TYPE_LKP['type_group_text'] = PLAY_TYPE_LKP['type_group_text'].astype(str)
PLAY_TYPE_LKP['type_text'] = PLAY_TYPE_LKP['type_text'].astype(str)

PLAY_INFO_OVERVIEW = play_overview.copy()
PLAY_INFO_OVERVIEW['id'] = PLAY_INFO_OVERVIEW['id'].astype(int)
PLAY_INFO_OVERVIEW['team_id'] = PLAY_INFO_OVERVIEW['team_id'].fillna(0)
PLAY_INFO_OVERVIEW['team_id'] = PLAY_INFO_OVERVIEW['team_id'].astype(int)

ATHLETE_INFO_LKP = player_lkp.copy()
ATHLETE_INFO_LKP['athlete_display_name'] = ATHLETE_INFO_LKP['athlete_display_name'].astype(str)
ATHLETE_INFO_LKP['athlete_position_name'] = ATHLETE_INFO_LKP['athlete_position_name'].astype(str)
ATHLETE_INFO_LKP['athlete_position_abbreviation'] = ATHLETE_INFO_LKP['athlete_position_abbreviation'].astype(str)

ATHLETE_ACTIVE_TEAM_LKP = player_team_lkp.copy()
ATHLETE_ACTIVE_TEAM_LKP['first_game_date'] = pd.to_datetime(ATHLETE_ACTIVE_TEAM_LKP['first_game_date'])
ATHLETE_ACTIVE_TEAM_LKP['first_game_date'] = ATHLETE_ACTIVE_TEAM_LKP['first_game_date'].dt.date
ATHLETE_ACTIVE_TEAM_LKP['last_game_date'] = pd.to_datetime(ATHLETE_ACTIVE_TEAM_LKP['last_game_date'])
ATHLETE_ACTIVE_TEAM_LKP['last_game_date'] = ATHLETE_ACTIVE_TEAM_LKP['last_game_date'].dt.date

PLAYER_GAME_STATS = player_game_stat.copy()
PLAYER_GAME_STATS['plus_minus'] = PLAYER_GAME_STATS['plus_minus'].astype(str)
PLAYER_GAME_STATS['reason'] = PLAYER_GAME_STATS['reason'].astype(str)

TEAM_INFO_LKP = team_lkp.copy()

GAME_INFO_OVERVIEW = final_game_overview.copy()
GAME_INFO_OVERVIEW['game_date'] =  pd.to_datetime(GAME_INFO_OVERVIEW['game_date'])
GAME_INFO_OVERVIEW['game_date'] =  GAME_INFO_OVERVIEW['game_date'].dt.date

GAME_INFO_OVERVIEW['game_date_time'] = pd.to_datetime(GAME_INFO_OVERVIEW['game_date'])

HOME_TEAM_GAME_STATS = home_stats.copy()
HOME_TEAM_GAME_STATS['fast_break_points'] = HOME_TEAM_GAME_STATS['fast_break_points'].astype(int)

AWAY_TEAM_GAME_STATS = away_stats.copy()
AWAY_TEAM_GAME_STATS['fast_break_points'] = AWAY_TEAM_GAME_STATS['fast_break_points'].astype(int)


In [65]:
# Sending tables to parquet files

GAME_PERIOD_LKP.to_parquet('./data/GAME_PERIOD_LKP.parquet', engine='pyarrow')
PERIOD_INFO_LKP.to_parquet('./data/PERIOD_INFO_LKP.parquet', engine='pyarrow')
GAME_PLAY_SCORE_LKP.to_parquet('./data/GAME_PLAY_SCORE_LKP.parquet', engine='pyarrow')
PLAY_TYPE_LKP.to_parquet('./data/PLAY_TYPE_LKP.parquet', engine='pyarrow')
PLAY_INFO_OVERVIEW.to_parquet('./data/PLAY_INFO_OVERVIEW.parquet', engine='pyarrow')
ATHLETE_INFO_LKP.to_parquet('./data/ATHLETE_INFO_LKP.parquet', engine='pyarrow')
ATHLETE_ACTIVE_TEAM_LKP.to_parquet('./data/ATHLETE_ACTIVE_TEAM_LKP.parquet', engine='pyarrow')
PLAYER_GAME_STATS.to_parquet('./data/PLAYER_GAME_STATS.parquet', engine='pyarrow')
TEAM_INFO_LKP.to_parquet('./data/TEAM_INFO_LKP.parquet', engine='pyarrow')
GAME_INFO_OVERVIEW.to_parquet('./data/GAME_INFO_OVERVIEW.parquet', engine='pyarrow')
HOME_TEAM_GAME_STATS.to_parquet('./data/HOME_TEAM_GAME_STATS.parquet', engine='pyarrow')
AWAY_TEAM_GAME_STATS.to_parquet('./data/AWAY_TEAM_GAME_STATS.parquet', engine='pyarrow')


In [ ]:
## ERD FOUND HERE:
# https://lucid.app/lucidchart/1caef344-3012-4512-8dc9-6baeb6668188/edit?viewport_loc=1166%2C-6780%2C2985%2C1600%2C0_0&invitationId=inv_a41b97dc-1c1b-473f-8194-62472895adc6

## Testing Analysis and Visuals

In [ ]:
# duck.read_parquet('')
query = """
with q1 as (
    select a1.game_id
        , a1.home_team_id
        , a2.team_score as home_team_score
        , a2.team_winner as home_team_winner
        , a3.team_display_name as home_team_display_name
        , a3.team_logo as home_team_logo
    from './data/GAME_INFO_OVERVIEW.parquet' a1
    inner join './data/HOME_TEAM_GAME_STATS.parquet' a2
    on a1.game_id = a2.game_id and a1.home_team_id=a2.team_id
    left join './data/TEAM_INFO_LKP.parquet' a3
    on a1.home_team_id = a3.team_id
),

q2 as (
    select a1.game_id
        , a1.away_team_id
        , a2.team_score as away_team_score
        , a2.team_winner as away_team_winner
        , a3.team_display_name as away_team_display_name
        , a3.team_logo as away_team_logo
    from './data/GAME_INFO_OVERVIEW.parquet' a1
    inner join './data/AWAY_TEAM_GAME_STATS.parquet' a2
    on a1.game_id = a2.game_id and a1.away_team_id=a2.team_id
    left join './data/TEAM_INFO_LKP.parquet' a3
    on a1.away_team_id = a3.team_id
)

select
    t1.game_id
    , t1.game_date
    , t1.game_title
    , q1.home_team_id
    , q1.home_team_score
    , q1.home_team_winner
    , q1.home_team_display_name
    , q1.home_team_logo 
    , q2.away_team_id
    , q2.away_team_score
    , q2.away_team_winner
    , q2.away_team_display_name
    , q2.away_team_logo
from './data/GAME_INFO_OVERVIEW.parquet' t1
left join q1
on q1.game_id = t1.game_id
left join q2
on q2.game_id = t1.game_id

"""

# Execute query with DuckDB
game_matchup_view = duck.query(query).df()

print(game_matchup_view.shape)
game_matchup_view.head()

In [ ]:
query = """
with q1 as (
    select *
    from './data/HOME_TEAM_GAME_STATS.parquet'
    UNION
    select *
    from './data/AWAY_TEAM_GAME_STATS.parquet'
)
select q1.*
    , a1.team_display_name
    , a2.game_date
from q1
left join './data/TEAM_INFO_LKP.parquet' a1
on q1.team_id = a1.team_id
left join './data/GAME_INFO_OVERVIEW.parquet' a2
on q1.game_id = a2.game_id
order by q1.game_id
"""

# Can then use these to calculate averages, other stats
team_game_stats_view = duck.query(query).df()

print(team_game_stats_view.shape)
team_game_stats_view.head()

In [ ]:
query = """
with sub as (
    select q1.*
        , a1.game_title
        , a1.game_date
        , a2.athlete_display_name
        , a2.athlete_headshot_href
    from './data/PLAYER_GAME_STATS.parquet' q1
    left join './data/GAME_INFO_OVERVIEW.parquet' a1
    on q1.game_id = a1.game_id
    left join './data/ATHLETE_INFO_LKP.parquet' a2
    on q1.athlete_id = a2.athlete_id
    order by q1.game_id
),

sub2 as (
select sub.*
    , t1.team_id
    , t1.first_game_date
    , t1.last_game_date
from sub
left join './data/ATHLETE_ACTIVE_TEAM_LKP.parquet' t1
on sub.athlete_id = t1.athlete_id
),

sub3 as (
    select sub2.*
    , case when sub2.game_date between sub2.first_game_date and sub2.last_game_date then sub2.team_id
    else null end as active_team_id
    from sub2
),

sub4 as (
    select sub3.*
    from sub3
    where sub3.active_team_id is not null
)

select sub4.*
    , l1.team_display_name
from sub4
left join './data/TEAM_INFO_LKP.parquet' l1
on sub4.active_team_id = l1.team_id
"""

# getting player game stats with active team
player_game_stats_view = duck.query(query).df()

print(player_game_stats_view.shape)
player_game_stats_view.head()

In [ ]:
query = """
with q1 as (
    select sub.* 
        , sub2.clock_display_value
        , sub2.id
        , sub3.game_date
        , sub3.game_title
    from './data/GAME_PERIOD_LKP.parquet' sub
    left join './data/PERIOD_INFO_LKP.parquet' sub2
    on sub.period_id = sub2.period_id
    left join './data/GAME_INFO_OVERVIEW.parquet' sub3
    on sub.game_id = sub3.game_id
)

select t1.id
    , t1.type_id
    , t1.team_id
    , l1.type_text
    , l1.type_group_text
    , t1.scoring_play
    , t1.score_value
    , t1.athlete_id_1
    , t1.coordinate_x
    , t1.coordinate_y
    , a2.athlete_display_name
    , q1.clock_display_value
    , q1.game_date
    , q1.game_title
    , q1.period_number
from './data/PLAY_INFO_OVERVIEW.parquet' t1
left join './data/ATHLETE_INFO_LKP.parquet' a2
on t1.athlete_id_1 = a2.athlete_id
left join q1
on t1.id = q1.id
left join './data/PLAY_TYPE_LKP.parquet' l1
on t1.type_id = l1.type_id
where t1.shooting_play = True
and l1.type_group_text not in ('free-throw')
"""

# Can then use these to calculate averages, other stats
shooting_chart_view = duck.query(query).df()

print(shooting_chart_view.shape)
shooting_chart_view.head()

### Shot Chart

In [ ]:
testing_chart = shooting_chart_view.copy()
testing_chart = testing_chart[testing_chart['athlete_display_name'] == 'Luka Doncic'].reset_index(drop=True)
testing_chart['quarter_time_clock'] = "Quarter " + testing_chart['period_number'].astype(str) + " - " + testing_chart['clock_display_value'].astype(str)
testing_chart['game_date_frmt'] = testing_chart['game_date'].dt.strftime('%B %d, %Y')

testing_chart.head()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=testing_chart['coordinate_x'],
                            y=testing_chart['coordinate_y'],
                            mode='markers',
                            marker=dict(
                                line=dict(width=2, color='DarkSlateGrey'),
                                symbol=[ 'circle' if x > 0 else 'x' for x in testing_chart['score_value']],
                                color='#FB4141',
                                size=8,
                            ),
                            text=testing_chart['score_value'],
                            customdata=testing_chart[['game_date_frmt', 'game_title', 'quarter_time_clock', 'score_value', 'type_text']],
                            hovertemplate = '<b>Game Date:</b> %{customdata[0]}<br>' + 
                            '<b>Game Title: </b>%{customdata[1]}<br>' 
                            + '<b>Shot Time:</b> %{customdata[2]}<br>'
                            + '<b>Shot Points:</b> %{customdata[3]}<br>'
                            + '<b>Shot Type:</b> %{customdata[4]}<br> <extra></extra>' 
                            ))


fig.add_layout_image(
        dict(
            source="./data/court.jpg",
            x=0,
            y=1,
            xref="paper",
            yref="paper",
            sizex=1,
            sizey=1,
            xanchor="left",
            yanchor="top",
            sizing="stretch",
            layer="below" )
)


fig.update_yaxes(visible=False, showticklabels=False)
fig.update_xaxes(visible=False, showticklabels=False)


# Set templates

fig.update_layout(
                plot_bgcolor='#111111',
                template='plotly_dark',
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=16,
                    font_family="Courier New, monospace"
                ),
                margin=dict(l=20, r=20, t=60, b=20),
                font=dict(family="Courier New, monospace"),
                xaxis=dict(zeroline=False, showgrid=False),
                yaxis=dict(zeroline=False, showgrid=False),)

fig.show(config={'displayModeBar': False, 'doubleClickDelay': 1000})

### Roster Lookup Chart

In [ ]:
# 401585475

roster_chart = player_game_stats_view.copy()
roster_chart = roster_chart[(roster_chart['game_id'] == 401585475) &
                            (roster_chart['team_display_name'] == 'Sacramento Kings') & 
                            (roster_chart['did_not_play'] == False)].reset_index(drop=True)
roster_chart['plus_minus_frmt'] = pd.to_numeric(roster_chart['plus_minus'], errors='coerce').astype('Int64')
roster_chart['game_date_frmt'] = roster_chart['game_date'].dt.strftime('%B %d, %Y')

roster_chart['fg_frmt'] = ((((roster_chart['field_goals_made'] / roster_chart['field_goals_attempted'])*100).round(0)).astype(int)).astype(str) \
                        + "%  (" + roster_chart['field_goals_made'].astype(int).astype(str) + "/" + roster_chart['field_goals_attempted'].astype(int).astype(str) + ")"


roster_chart['3fg_frmt'] = roster_chart.apply(
                                        lambda x: str(int((x['three_point_field_goals_made'] / x['three_point_field_goals_attempted'])*100)) + \
                                            "%  (" + str(int(x['three_point_field_goals_made'])) + "/" + str(int(x['three_point_field_goals_attempted'])) + \
                                            ")" if x['three_point_field_goals_attempted'] > 0 else "0%", axis=1
                                        )

roster_chart['ft_frmt'] = roster_chart.apply(
                                        lambda x: str(int((x['free_throws_made'] / x['free_throws_attempted'])*100)) + \
                                            "%  (" + str(int(x['free_throws_made'])) + "/" + str(int(x['free_throws_attempted'])) + \
                                            ")" if x['free_throws_attempted'] > 0 else "0%", axis=1
                                        )

print(roster_chart.shape)
roster_chart.head()

In [ ]:
roster_chart.columns

In [ ]:
import urllib.request
from io import BytesIO 
from PIL import Image
import io
import base64


fig = go.Figure()

fig.add_trace(
    go.Scatter(x=roster_chart['minutes'],
                            y=roster_chart['plus_minus_frmt'],
                            mode='markers',
                            marker=dict(
                                line=dict(width=2, color='DarkSlateGrey'),
                                # symbol=[ 'circle' if x > 0 else 'x' for x in roster_chart['score_value']],
                                color='#FB4141',
                                size=8,
                                opacity=0
                            ),
                            customdata=roster_chart[['athlete_display_name', 'minutes', 'fg_frmt', '3fg_frmt', 'ft_frmt', 
                                                    'rebounds', 'assists',
                                                    'steals', 'blocks', 'turnovers', 'fouls', 'plus_minus', 'points',
                                                    'starter']],
                            hovertemplate = '<b>Player Name: </b>%{customdata[0]}<br>' 
                            + '<b>Minutes Played: </b>%{customdata[1]}<br>' 
                            + '<b>FG %:</b> %{customdata[2]}<br>'
                            + '<b>3-PT FG %:</b> %{customdata[3]}<br>'
                            + '<b>Free-Throw %:</b> %{customdata[4]}<br>'
                            + '<b>Rebounds:</b> %{customdata[5]}<br>'
                            + '<b>Assists:</b> %{customdata[6]}<br>'
                            + '<b>Steals:</b> %{customdata[7]}<br>'
                            + '<b>Blocks:</b> %{customdata[8]}<br>'
                            + '<b>Turnovers:</b> %{customdata[9]}<br>'
                            + '<b>Fouls:</b> %{customdata[10]}<br>'
                            + '<b>Plus-Minus:</b> %{customdata[11]}<br>'
                            + '<b>Total Points:</b> %{customdata[12]}<br>'
                            + '<b>Starter?:</b> %{customdata[13]}<br> <extra></extra>' 
                            ))

for t in roster_chart["athlete_display_name"].unique():
    url = roster_chart.loc[roster_chart['athlete_display_name']== f"{t}", "athlete_headshot_href"].values[0]
    # Opening image
    img = Image.open(BytesIO(urllib.request.urlopen(url).read())) 

    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    encoded_image = base64.b64encode(buffer.getvalue()).decode("utf-8")
    data_uri = f"data:image/png;base64,{encoded_image}"

    fig.add_layout_image(
        dict(
        source=data_uri,
        sizex=9,
        sizey=9,
        name=t,
        xref="x",
        yref="y",
        x=int(roster_chart.loc[roster_chart['athlete_display_name']== f"{t}", "minutes"].values[0]),
        y=int(roster_chart.loc[roster_chart['athlete_display_name']== f"{t}", "plus_minus_frmt"].values[0]),
        layer="above",
        opacity=.8, 
        xanchor="center",
        yanchor="middle",
        sizing="contain"
    ))

    # fig.add_shape(
    #     type="circle",
    #     x0=int(roster_chart.loc[roster_chart['athlete_display_name']== f"{t}", "minutes"].values[0]) - 1, 
    #     x1=int(roster_chart.loc[roster_chart['athlete_display_name']== f"{t}", "minutes"].values[0]) + 1,
    #     y0=int(roster_chart.loc[roster_chart['athlete_display_name']== f"{t}", "plus_minus_frmt"].values[0]) - 6.5,
    #     y1=int(roster_chart.loc[roster_chart['athlete_display_name']== f"{t}", "plus_minus_frmt"].values[0]) + 4,
    #     xref="x",
    #     yref="y",
    #     line=dict(color="white", width=2),
    #     fillcolor="#419197",
    # )

fig.update_shapes(opacity=0.2)

fig.update_layout(
                plot_bgcolor='#111111',
                template='plotly_dark',
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=16,
                    font_family="Courier New, monospace"
                ),
                margin=dict(l=20, r=20, t=60, b=20),
                font=dict(family="Courier New, monospace"),
                xaxis_title='Minutes Played',
                yaxis_title='Plus-Minus',
                xaxis=dict(zeroline=True, showgrid=False),
                yaxis=dict(zeroline=True, showgrid=False),
                yaxis_range=[int(roster_chart['plus_minus_frmt'].min())-10, int(roster_chart['plus_minus_frmt'].max())+10],
                )


fig.show(config={'displayModeBar': False, 'doubleClickDelay': 1000})

### Moving Average

In [79]:
def calculate_moving_average(data, window_size):
    df = pd.DataFrame(data, columns=['Actual'])
    moving_average = df['Actual'].rolling(window=window_size).mean()
    return moving_average

In [88]:
def mov_avg_game(player_name, df, attr):
    init_df = df.sort_values(['athlete_display_name', 'game_date']).reset_index(drop=True)
    fltr_data = init_df[init_df['athlete_display_name']==player_name].reset_index(drop=True)

    data_points = fltr_data[attr].values.tolist()
    moving_avg_3_days = calculate_moving_average(data_points, window_size=3)
    moving_avg_5_days = calculate_moving_average(data_points, window_size=5)
    moving_avg_7_days = calculate_moving_average(data_points, window_size=7)

    fltr_data['mov_avg_3_gms'] = moving_avg_3_days
    fltr_data['mov_avg_5_gms'] = moving_avg_5_days
    fltr_data['mov_avg_7_gms'] = moving_avg_7_days

    actual_dp = fltr_data[attr].values.tolist()
    moving_avg_3= fltr_data["mov_avg_3_gms"].values.tolist()
    moving_avg_5 = fltr_data["mov_avg_5_gms"].values.tolist()
    moving_avg_7 = fltr_data["mov_avg_7_gms"].values.tolist()

    fig = go.Figure()

    # Add trace for actual points
    fig.add_trace(go.Scatter(x=list(range(1, len(actual_dp)+1))
                            , y=actual_dp, mode='lines+markers', name='Actual Points'
                            # , text=text_points
                            , marker=dict(
                                color='#419197',
                                size=5,
                                # angleref="previous"
                            ),
                            customdata=fltr_data[['game_date', 'game_title']],
                            hovertemplate = '<b>Game Date:</b> %{customdata[0]}<br>' + 
                            '<b>Game Matchup: </b>%{customdata[1]}<br>' + '<b>Attribute:</b> %{y}<br>'
                            ))

    # Add trace for moving average
    fig.add_trace(go.Scatter(x=list(range(1, len(moving_avg_3)+1))
                            , y=moving_avg_3, mode='lines', name='3-Game Moving Average'
                            , marker=dict(
                                color='#FF1E1E'
                            ),
                            hovertemplate = '<b>3-GM MA:</b> %{y}<br>'))

    fig.add_trace(go.Scatter(x=list(range(1, len(moving_avg_5)+1))
                            , y=moving_avg_5, mode='lines', name='5-Game Moving Average'
                            , marker=dict(
                                color='#06D001'
                            ),
                            hovertemplate = '<b>5-GM MA:</b> %{y}<br>'))

    fig.add_trace(go.Scatter(x=list(range(1, len(moving_avg_7)+1))
                            , y=moving_avg_7, mode='lines', name='7-Game Moving Average'
                            , marker=dict(
                                color='#FDFF00'
                            ),
                            hovertemplate = '<b>7-GM MA:</b> %{y}<br>'))

    fig.update_annotations(font_size=9, bordercolor='#419197', font_family="Courier New, monospace", font_color='black', borderwidth=1.5, borderpad=1.5, bgcolor='white')

    # Update layout
    fig.update_layout(
                    title=dict(
                        text="Game Trends",
                        xanchor='left',
                        yanchor='top'     
                    ),
                    xaxis_title='Game number',
                    yaxis_title=attr,
                    plot_bgcolor='#111111',
                    template='plotly_dark',
                    hoverlabel=dict(
                    bgcolor="white",
                    font_size=16,
                    font_family="Courier New, monospace"),
                    margin=dict(l=20, r=20, t=60, b=20),
                    font=dict(family="Courier New, monospace"))

    fig.update_yaxes(showline=False, gridcolor='#7D7C7C', griddash='dash', showspikes=True)
    fig.update_xaxes(showline=False, gridcolor='#7D7C7C', griddash='dash')
    # Show the figure
    # fig.show(config={'displayModeBar': False, 'doubleClickDelay': 1000})

    return fig

In [ ]:
ma_attr_chart = player_game_stats_view.copy()
ma_attr_chart = ma_attr_chart[(ma_attr_chart['athlete_display_name'] == 'Luka Doncic') & 
                            (ma_attr_chart['did_not_play'] == False)].reset_index(drop=True)


ma_attr_chart.head()

In [ ]:
mov_avg_game('Luka Doncic', ma_attr_chart, 'points')